In [ ]:
import time
from ase.io import read
from sklearn.manifold import TSNE
import chemiscope
import numpy as np
import os
from dscribe.descriptors import SOAP
from mace.calculators.foundations_models import mace_mp
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Start timing
start_time = time.time()

# Load dataset
a = read('li_MP_li_MatGen_gen_data.db', index=':')

def soap_tnse_with_environments(frames, environments):
    if environments is None:
        raise ValueError("'environments' must be provided")

    grouped_envs = {}
    unique_structures = set()

    # Get atom-centered indices from environments
    for [env_index, atom_index, _cutoff] in environments:
        if env_index not in grouped_envs:
            grouped_envs[env_index] = []
        grouped_envs[env_index].append(atom_index)
        unique_structures.add(env_index)
    centers = list(grouped_envs.values())

    # only include frames that are present in the environments
    if len(unique_structures) != len(frames):
        frames = [frames[index] for index in sorted(unique_structures)]

    # Get global species
    species = set()
    for frame in frames:
        species.update(frame.get_chemical_symbols())
    species = list(species)

    # Initialize calculator
    soap = SOAP(
        species=species,
        r_cut=4.5,
        n_max=8,
        l_max=6,
        sigma=0.2,
        rbf="gto",
        average="outer",
        periodic=True,
        weighting={"function": "pow", "c": 1, "m": 5, "d": 1, "r0": 3.5},
        compression={"mode": "mu1nu1"},
    )

    # Calculate descriptors
    feats = soap.create(frames, centers=centers)

    # Compute tsne
    perplexity = min(50, feats.shape[0] - 1)
    reducer = TSNE(n_components=2, perplexity=perplexity)
    return reducer.fit_transform(feats)

cs = chemiscope.explore(
    frames=a,
    featurize=soap_tnse_with_environments,
    environments=chemiscope.all_atomic_environments(a),
)

cs.save("SOAP_li_MP_li_gen_data.html")

# End timing
end_time = time.time()
elapsed = end_time - start_time
print(f"\n Done! Total time: {elapsed:.2f} seconds.")